In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv')
df = df.drop_duplicates()

X = df.drop("purchaseValue",axis=1)
Y = df["purchaseValue"]

In [ ]:
X_predict = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv')

In [ ]:
print(list(X.columns))

In [ ]:
not_available_in_demo = ['device.screenResolution','device.mobileDeviceBranding','device.mobileInputSelector','device.mobileDeviceMarketingName',
                         'device.operatingSystemVersion','device.flashVersion','geoNetwork.networkLocation','browserMajor','device.browserSize',
                         'device.mobileDeviceModel','device.language','device.browserVersion','device.screenColors']
pk = ['userId','sessionId']
single_value = ['screenSize','totals.visits','socialEngagementType','locationZone']

categorical = ['browser','geoCluster','trafficSource.adwordsClickInfo.slot','geoNetwork.networkDomain','os','geoNetwork.subContinent',
               'trafficSource.medium','trafficSource.adwordsClickInfo.adNetworkType','trafficSource.adwordsClickInfo.page','deviceType','userChannel'
               ,'geoNetwork.continent']
too_much_categorical = ['geoNetwork.region','trafficSource','locationCountry','geoNetwork.city','geoNetwork.metro','trafficSource.referralPath']
numerical = ['pageViews','totalHits','sessionNumber']
boolean = ['trafficSource.isTrueDirect','gclIdPresent','trafficSource.adwordsClickInfo.isVideoAd','totals.bounces','device.isMobile']
textual = ['trafficSource.adContent','trafficSource.keyword','trafficSource.campaign']
date = ['date']

In [ ]:
# print(sorted(list(df['purchaseValue'].value_counts().keys())))

In [ ]:
X_predict['trafficSource'].value_counts()

In [ ]:
print(df[['date','sessionStart']])

In [ ]:
print(df.shape)

In [ ]:
print(df['new_visits'].value_counts())

In [ ]:
print(df[df['new_visits']!=1.0]['purchaseValue'].mean()-df[df['new_visits']==1.0]['purchaseValue'].mean())

In [ ]:
print(df[['geoNetwork.continent','geoNetwork.subContinent']].value_counts())

In [ ]:
print(df[df['totals.bounces']==1.0]['purchaseValue'].value_counts())

In [ ]:
req_cols = categorical+too_much_categorical+numerical+boolean+textual+date+['sessionStart']
X = X[req_cols]
X_predict = X_predict[req_cols]

In [ ]:
# df['trafficSource.adwordsClickInfo.page'].value_counts()
# X['tfsrcinfopage1'] = (X['trafficSource.adwordsClickInfo.page']==1).astype(int)
# X['tfsrcinfopage0'] = (X['trafficSource.adwordsClickInfo.page'].isna()).astype(int)
# X_predict['tfsrcinfopage1'] = (X_predict['trafficSource.adwordsClickInfo.page']==1).astype(int)
# X_predict['tfsrcinfopage0'] = (X_predict['trafficSource.adwordsClickInfo.page'].isna()).astype(int)

In [ ]:
X.shape

In [ ]:
X_predict.shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df['purchaseValue'],df['pageViews'])
plt.xlabel('purchaseValue')
plt.grid(True)
plt.show()


In [ ]:
plt.plot(df['purchaseValue'],df['totalHits'])
plt.xlabel('purchaseValue')
plt.grid(True)
plt.show()

In [ ]:
X['date'] = pd.to_datetime(X['date'], format='%Y%m%d')
X_predict['date'] = pd.to_datetime(X_predict['date'], format='%Y%m%d')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
date_counts1 = X['date'].value_counts().sort_index()
date_counts2 = X_predict['date'].value_counts().sort_index()

plt.plot(date_counts1.index, date_counts1.values, label='train')
plt.plot(date_counts2.index, date_counts2.values, label='predict')

plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Frequency of Each Date')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
for elem in [X,X_predict]:
    elem['year'] = elem['date'].dt.year
    elem['month'] = elem['date'].dt.month
    elem['dayofweek'] = elem['date'].dt.dayofweek
    elem['is_weekend'] = elem['dayofweek'] >= 5
    elem.drop(['date'],inplace=True,axis=1)
    elem['dow_sin'] = np.sin(2 * np.pi * elem['dayofweek'] / 7)
    elem['dow_cos'] = np.cos(2 * np.pi * elem['dayofweek'] / 7)
    elem['month_sin'] = np.sin(2 * np.pi * elem['month'] / 12)
    elem['month_cos'] = np.cos(2 * np.pi * elem['month'] / 12)
    
date.clear()
date = ['year','month','is_weekend']
numerical += date
categorical += ['dayofweek']
date.clear()

In [ ]:
print(X['year'].value_counts())

In [ ]:
pd.to_datetime(X['sessionStart'],unit='s')

In [ ]:
for elem in [X,X_predict]:
    elem['sessionStart'] = pd.to_datetime(elem['sessionStart'], unit='s')
    elem['hour_session'] = elem['sessionStart'].dt.hour # ise one hot encode kr lunga
    elem['dayofweek_session'] = elem['sessionStart'].dt.dayofweek
    elem['is_weekend_session'] = elem['dayofweek_session'].isin([5, 6]).astype(int)
    elem['isnight_session'] = elem['hour_session'].apply(lambda h: 1 if h < 6 or h >= 22 else 0)
    elem['hour_sin'] = np.sin(2 * np.pi * elem['hour_session'] / 24)
    elem['hour_cos'] = np.cos(2 * np.pi * elem['hour_session'] / 24)
    elem['is_weekend_night_session'] = elem['is_weekend_session']*elem['isnight_session']
    elem['dow_sin_session'] = np.sin(2 * np.pi * elem['dayofweek_session'] / 7)
    elem['dow_cos_session'] = np.cos(2 * np.pi * elem['dayofweek_session'] / 7)
    elem = elem.drop(['sessionStart'],axis=1)

In [ ]:
print(X.columns)

In [ ]:
d1 = (X['browser'].value_counts())
d2 = (X_predict['browser'].value_counts())
d1 = dict(d1)
d2 = dict(d2)
for i in d1:
    if i not in d2:
        X['browser'] = X['browser'].replace(i,'(not set)')

for i in d2:
    if i not in d1:
        X_predict['browser'] = X_predict['browser'].replace(i,'(not set)')

In [ ]:
d1 = (X['os'].value_counts())
d2 = (X_predict['os'].value_counts())
d1 = dict(d1)
d2 = dict(d2)
for i in d1:
    if i not in d2:
        X['os'] = X['os'].replace(i,'(not set)')

for i in d2:
    if i not in d1:
        X_predict['os'] = X_predict['os'].replace(i,'(not set)')

In [ ]:
# d1 = (X['trafficSource'].value_counts())
# d2 = (X_predict['trafficSource'].value_counts())
# d1 = dict(d1)
# d2 = dict(d2)
# for i in d1:
#     if i not in d2:
#         X['trafficSource'] = X['trafficSource'].replace(i,'(not set)')

# for i in d2:
#     if i not in d1:
#         X_predict['trafficSource'] = X_predict['trafficSource'].replace(i,'(not set)')

In [ ]:
# pattern = r"\{KeyWord:(.*?)\}"
# X['trafficSource.adContent'] = X['trafficSource.adContent'].astype(str).str.replace(pattern, r"\1", regex=True)
# X_predict['trafficSource.adContent'] = X_predict['trafficSource.adContent'].astype(str).str.replace(pattern, r"\1", regex=True)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer(max_features=100)
# X_tfidf = tfidf.fit_transform(X['trafficSource.adContent'])

# tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=[f"tfidf_{word}" for word in tfidf.get_feature_names_out()])
# X = pd.concat([X.drop(columns=['trafficSource.adContent']).reset_index(drop=True),tfidf_df.reset_index(drop=True)], axis=1)

# X_tfidf = tfidf.transform(X_predict['trafficSource.adContent'])
# tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=[f"tfidf_{word}" for word in tfidf.get_feature_names_out()])
# X_predict = pd.concat([X_predict.drop(columns=['trafficSource.adContent']).reset_index(drop=True),tfidf_df.reset_index(drop=True)], axis=1)

In [ ]:
X['pageViews'] = X['pageViews'].fillna(X['pageViews'].mean())
X_predict['pageViews'] = X_predict['pageViews'].fillna(X['pageViews'].mean())

In [ ]:
X['trafficSource.adwordsClickInfo.adNetworkType'] = X['trafficSource.adwordsClickInfo.adNetworkType'].fillna('not set')
X_predict['trafficSource.adwordsClickInfo.adNetworkType'] = X_predict['trafficSource.adwordsClickInfo.adNetworkType'].fillna('not set')

In [ ]:
X['trafficSource.adwordsClickInfo.slot'] = X['trafficSource.adwordsClickInfo.slot'].fillna('unknown')
X_predict['trafficSource.adwordsClickInfo.slot'] = X_predict['trafficSource.adwordsClickInfo.slot'].fillna('unknown')
X['trafficSource.adwordsClickInfo.slot'] = X['trafficSource.adwordsClickInfo.slot'].replace('Google Display Network','unknown')

In [ ]:
X['trafficSource.isTrueDirect'] = X['trafficSource.isTrueDirect'].fillna(0)
X['trafficSource.isTrueDirect'] = X['trafficSource.isTrueDirect'].replace(True,1)
X['trafficSource.isTrueDirect'] = X['trafficSource.isTrueDirect'].astype(int)

X_predict['trafficSource.isTrueDirect'] = X_predict['trafficSource.isTrueDirect'].fillna(0)
X_predict['trafficSource.isTrueDirect'] = X_predict['trafficSource.isTrueDirect'].replace(True,1)
X_predict['trafficSource.isTrueDirect'] = X_predict['trafficSource.isTrueDirect'].astype(int)

In [ ]:
X['device.isMobile'] = X['device.isMobile'].replace(False,0)
X['device.isMobile'] = X['device.isMobile'].replace(True,1)
X['device.isMobile'] = X['device.isMobile'].astype(int)

X_predict['device.isMobile'] = X_predict['device.isMobile'].replace(False,0)
X_predict['device.isMobile'] = X_predict['device.isMobile'].replace(True,1)
X_predict['device.isMobile'] = X_predict['device.isMobile'].astype(int)

In [ ]:
X['trafficSource.adwordsClickInfo.isVideoAd'] = X['trafficSource.adwordsClickInfo.isVideoAd'].fillna(1)
X['trafficSource.adwordsClickInfo.isVideoAd'] = X['trafficSource.adwordsClickInfo.isVideoAd'].replace(False,0)
X['trafficSource.adwordsClickInfo.isVideoAd'] = X['trafficSource.adwordsClickInfo.isVideoAd'].astype(int)

X_predict['trafficSource.adwordsClickInfo.isVideoAd'] = X_predict['trafficSource.adwordsClickInfo.isVideoAd'].fillna(1)
X_predict['trafficSource.adwordsClickInfo.isVideoAd'] = X_predict['trafficSource.adwordsClickInfo.isVideoAd'].replace(False,0)
X_predict['trafficSource.adwordsClickInfo.isVideoAd'] = X_predict['trafficSource.adwordsClickInfo.isVideoAd'].astype(int)

In [ ]:
X['geoNetwork.region'].value_counts()

In [ ]:
X_predict['geoNetwork.region'].value_counts()

In [ ]:
X.info()

In [ ]:
onehot = ['browser','geoCluster','os','userChannel','deviceType','dayofweek','geoNetwork.subContinent',
          'trafficSource.medium','trafficSource.adwordsClickInfo.adNetworkType','geoNetwork.networkDomain'
          ,'trafficSource.adwordsClickInfo.slot','locationCountry','dayofweek_session'] #hour_session 

numerical = ['pageViews', 'totalHits', 'year', 'month', 'is_weekend','trafficSource.isTrueDirect',
             'gclIdPresent','device.isMobile','trafficSource.adwordsClickInfo.isVideoAd','sessionNumber',
             'isnight_session','is_weekend_session','month_sin','month_cos','dow_sin','dow_cos','hour_sin','hour_cos','is_weekend_night_session'
            ,'dow_sin_session','dow_cos_session']

d = {'totals.bounces':1.0,'trafficSource.adwordsClickInfo.page':[2.0,3.0,4.0,5.0]} 

In [ ]:
from sklearn.model_selection import train_test_split
submit = True
if submit:
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.00000861898,random_state=1000)
    # X_train,Y_train = X,Y
else:
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=1000)

In [ ]:
X_train[onehot+numerical].shape

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.ensemble import ExtraTreesRegressor

condition = (X_train['totals.bounces'] == 1.0) | (X_train['trafficSource.adwordsClickInfo.page'].isin([2.0, 3.0, 4.0, 5.0]))

numpipeline = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore',sparse_output=False), onehot),
        ('num', numpipeline, numerical)
    ],
    remainder='passthrough'
)

# i could try chaging the value of k in select k best.
model = Pipeline(steps=[
    ('preprocessing', preprocessor),    
    ('select', SelectKBest(f_regression, k=30)),
    # ('regressor', HistGradientBoostingRegressor()) # 0.11,0.36
    # ('regressor',lgb.LGBMRegressor(n_estimators=1000,learning_rate=0.05,max_depth=7,random_state=42)) # 0.25,0.46
    # ('regressor',DecisionTreeRegressor(max_depth=9,min_samples_split=10,min_samples_leaf=5,max_leaf_nodes=31 )) # this has weird score, look into it
    # ('regressor',RandomForestRegressor(n_estimators=100, random_state=42)) # 0.575 test score, sus, but gotta try submitting it 
    # ('regressor',LGBMRegressor(n_estimators=20000,learning_rate=0.06,max_depth=None,num_leaves=127,colsample_bytree=0.7,subsample=0.7,random_state=42,n_jobs=-1)) # lowered the training, but same test
    # ('regressor',RandomForestRegressor(n_estimators=2500,max_depth=None,min_samples_split=2,min_samples_leaf=1,max_features=0.5,bootstrap=True,n_jobs=-1, random_state=42)) # 0.41,0.576
    ('regressor', ExtraTreesRegressor(n_estimators=2500,max_features=0.8,min_samples_split=2,min_samples_leaf=1, bootstrap=False,n_jobs=-1,random_state=42,verbose=1))
    # ('regressor',XGBRegressor(n_estimators=3853,learning_rate=0.11495493205167782,max_depth=9,colsample_bytree=0.5115312125207079,subsample=0.6163856702151521,reg_lambda=0.9737555188414592,reg_alpha=0.04666566321361543))
    # ('regressor', XGBRegressor(n_estimators=3000,learning_rate=0.05,max_depth=7,subsample=0.8,colsample_bytree=0.8,reg_alpha=0.1,reg_lambda=1.0, n_jobs=-1,random_state=42,verbosity=1))
])
# param_distributions = {
#     'regressor__n_estimators': randint(100, 4000),
#     'regressor__max_depth': randint(3, 10),
#     'regressor__learning_rate': uniform(0.01, 0.2),
#     'regressor__subsample': uniform(0.5, 0.5),
#     'regressor__colsample_bytree': uniform(0.5, 0.5),
#     'regressor__reg_alpha': uniform(0, 1),
#     'regressor__reg_lambda': uniform(0, 1),
# }

# cvmodel = RandomizedSearchCV(model,param_distributions=param_distributions,n_iter=5,cv=3,scoring='r2',verbose=2,random_state=54,n_jobs=-1)

# cvmodel.fit(X_train[onehot+numerical],Y_train)

# model = cvmodel

model.fit(X_train[~condition][onehot+numerical],Y_train[~condition])

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier,ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler,MaxAbsScaler

numpipeline = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
    # ('scaler',MaxAbsScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore',sparse_output=False), onehot),
        ('num', numpipeline, numerical)
    ],
    remainder='passthrough'
)

model1 = Pipeline(steps=[
    ('preprocessing', preprocessor),
    # ('feature_selection', SelectKBest(score_func=mutual_info_classif, k=30)),
    # ('classifier',XGBClassifier(n_estimators=2000,learning_rate=0.05,max_depth=7,subsample=0.8,colsample_bytree=0.8,reg_alpha=0.1,reg_lambda=1.0,use_label_encoder=False,n_jobs=-1,random_state=42))
    # ('classifier', RandomForestClassifier(n_estimators=1000,max_depth=None,max_features='sqrt',min_samples_split=2,min_samples_leaf=1,bootstrap=True,n_jobs=-1,random_state=42))
    ('classifier',LGBMClassifier(n_estimators=1500,learning_rate=0.05,max_depth=9,num_leaves=31,subsample=0.5,colsample_bytree=0.8,random_state=42,n_jobs=-1))
    # ('classifier',LGBMClassifier(random_state=42, n_jobs=-1))
    # ('classifier',XGBClassifier(n_estimators=1000,learning_rate=0.05,max_depth=7,subsample=0.8,colsample_bytree=0.8,reg_alpha=0.1,reg_lambda=1.0,use_label_encoder=False,eval_metric='logloss',random_state=42,n_jobs=-1))
    # ('classifier',ExtraTreesClassifier(n_estimators=2500,max_features='sqrt',max_depth=30,min_samples_split=2,min_samples_leaf=1,bootstrap=False,n_jobs=-1,random_state=42,verbose=1))
])

# param_dist = {
#     'classifier__n_estimators': randint(300, 2000),
#     'classifier__learning_rate': uniform(0.01, 0.2),
#     'classifier__max_depth': randint(3, 15),
#     'classifier__num_leaves': randint(20, 150),
#     'classifier__subsample': uniform(0.6, 0.4),
#     'classifier__colsample_bytree': uniform(0.6, 0.4),
#     'classifier__reg_alpha': uniform(0.0, 1.0),
#     'classifier__reg_lambda': uniform(0.0, 1.0)
# }

# search = RandomizedSearchCV(
#     model1,
#     param_distributions=param_dist,
#     n_iter=5,
#     cv=3,
#     scoring='accuracy',
#     verbose=2,
#     random_state=42,
#     n_jobs=-1
# )

# search.fit(X_train[onehot+numerical],(Y_train!=0).astype(int))
# print("Best score:", search.best_score_)
# print("Best params:", search.best_params_)
# model1 = search
model1.fit(X_train[onehot+numerical],(Y_train!=0).astype(int))

In [ ]:
Y_test_pred = model.predict(X_test[onehot+numerical])
class_test_pred = model1.predict(X_test[onehot+numerical])
# class_test_pred[class_test_pred<0.63] = 0
# class_test_pred[class_test_pred>=0.63] = 0.97
Y_test_pred = Y_test_pred * (class_test_pred)
condition = (X_test['totals.bounces'] == 1.0) | (X_test['trafficSource.adwordsClickInfo.page'].isin([2.0, 3.0, 4.0, 5.0]))
Y_test_pred[condition.to_numpy()] = 0
Y_test_pred = np.maximum(Y_test_pred,0)

Y_train_pred = model.predict(X_train[onehot+numerical])
class_train_pred = model1.predict(X_train[onehot+numerical])
# class_train_pred[class_train_pred<0.63] = 0
# class_train_pred[class_train_pred>=0.63] = 0.97
Y_train_pred = Y_train_pred * (class_train_pred)
condition = (X_train['totals.bounces'] == 1.0) | (X_train['trafficSource.adwordsClickInfo.page'].isin([2.0, 3.0, 4.0, 5.0]))
Y_train_pred[condition.to_numpy()] = 0
Y_train_pred = np.maximum(Y_train_pred, 0)

from sklearn.metrics import r2_score
print(r2_score(Y_train,Y_train_pred))
print(r2_score(Y_test,Y_test_pred))

# 0.9992014337836487 -> 40
# 0.6935053711233345

now i need to do something for overfitting thats it.

In [ ]:
Y_predict = model.predict(X_predict[onehot+numerical])
class_predict_pred = model1.predict(X_predict[onehot+numerical])
# class_predict_pred[class_predict_pred<0.63] = 0
# class_predict_pred[class_predict_pred>=0.63] = 0.97
Y_preidct = Y_predict * class_predict_pred
condition = (X_predict['totals.bounces'] == 1.0) | (X_predict['trafficSource.adwordsClickInfo.page'].isin([2.0, 3.0, 4.0, 5.0]))
Y_predict[condition] = 0
Y_predict = np.maximum(Y_predict, 0)

In [ ]:
Y = pd.DataFrame({'id': range(len(Y_predict)), 'purchaseValue': Y_predict})
Y.to_csv("/kaggle/working/submission.csv",index=False)